## How to use Desk
Desk is Franka Emika‘s web-based, intuitive and graphical programming and user interface.
* link: https://{PANDA-IP}/desk/

```Pilot mode``` [Tablet / End-effector]

* Tablet: control by apps on GUI
        drag & drop apps icon on blank area and save position, set velocity & acceleration of motion
        click 'Run the task' button to run teached motion 
* End-effector: control by buttons which located on top of the robot


```Guiding mode``` [Translation / Rotation / Free move / User]

* Translation: Only translatioal movement is possible on end-effector
* Rotation: Only rotational movement is possible on end-effector
* Rotation: Both rotational and translatioal movements are possible on end-effector
* User: User can restrict direction of movement on end-effector(X, Y, Z, R, P, Y)



```Joints``` [Lock / Unlock]

* Lock: lock all joints mechanically to make Panda cannot move 
* Unlock: Unlock the joints to make Panda can move

```Robot status```

* status of each part of Panda
* the status represented by color which is refered to below 'Operating mode'

## Operating mode
```WHITE [Interactive]```
* robot can move interacitively (direct teaching; push enabling button)

```BLUE [Activation]```

* recieve signal from PC and move based on the signal (pull enabling button)

```GREEN [Automatic execution]```
* carrying out an automatic program (moving from Desk)

```YELLOW [Lock]```
* joints are locked mechanically, so it cannot move (immediately after turn on Panda)

```PINK [Conflict]```

* occured conflict from get different operating mode signals simultaneously  

```RED [Error]```

* occured some errors

## How to implement 
```Controller (joint_control_rnb.launch)```
* see below cell

```Web UI```
* After executing joint_control_rnb.launch, control WebUI is on http://{PANDA-IP}:9990/
```
Select controller : change a controller
Set Default : set current controller as default
Apply : after change gains, press 'Apply' button
Save & Load : save current gain values & load saved values
Restart log & Pause log : start & pause log
Download full : save logged data as .csv file (only last 30 secs data can be saved)
```

```
Tip : before download saved data, pause log at the first
```
```Making trajectory (trajectroy_client)```
* follow this 'trajectory_client_example'

## Launch controller
* Execute below command after replacing {PANDA-IP} with the IP of Panda robot
```bash
roslaunch panda_control joint_control_rnb.launch robot_ip:={PANDA-IP} load_gripper:=false
```

* Tip
```
If an interruption (ex. error) occured on controller, push & pull enabling button and restart controller
```

## You need urdf_parser_py to use kinematics functions
```bash
pip install urdf_parser_py.py
```

In [ ]:
# ! pip install urdf_parser_py

## Initialize panda client

In [1]:
from utils.trajectory_client.trajectory_client import *
from utils.trajectory_client.panda_trajectory_client import PandaTrajectoryClient
traj_client = PandaTrajectoryClient(server_ip="192.168.21.14", robot_ip="192.168.21.13")

## Testing short motion

In [2]:
Q1 = traj_client.get_qcur()
Q2 = np.add(Q1, np.pi/36)

In [13]:
traj_client.move_joint_s_curve(Q2, N_div=50)

In [4]:
traj_client.move_joint_s_curve(Q1, N_div=50)

## direct teaching

In [5]:
qlist=[]

#### Set the controller to gravity compensator and run below cell multiple times to save waypoints
* You don't have to press enabling button for direct teaching

In [14]:
qlist.append(traj_client.get_qcur())
print("q: {}".format(qlist[-1]))


q: [0.0925, -0.2976, 0.1034, -1.5107, 0.073, 1.6572, 1.6582]


#### list all saved waypoints

In [9]:
for i_q, q in enumerate(qlist):
    print("q{}: {}".format(i_q, q))

q0: [0.0054, -0.3848, 0.0163, -1.5972, -0.0129, 1.5704, 1.5723]
q1: [0.0054, -0.3848, 0.0162, -1.5973, -0.0129, 1.5703, 1.5721]
q2: [0.0054, -0.3848, 0.0162, -1.5973, -0.013, 1.5703, 1.5721]


#### update target position and reset controller for safety

In [15]:
traj_client.move_joint_s_curve(traj_client.get_qcur(), N_div=50)
traj_client.reset()

{'step_c': 20}

#### Set controller to NRIC_PD or other active one  - if the robot moves abruptly, restart the controller program

#### Run below cell to repeat the waypoint motion (move_joint_s_curve version)

In [16]:
for _ in range(3):
    for q in qlist:
        traj_client.move_joint_s_curve(q, N_div=50) # adjust speed by N_div

#### Run below cell to repeat the waypoint motion (move_joint_wp version)

In [17]:
for _ in range(3):
    traj_client.move_joint_wp(qlist, vel_lims=0.5, acc_lims=0.5, auto_stop=True) # adjust speed by vel_lims and acc_lims

## Task trajectory generation

In [18]:
from urdf_parser_py.urdf import URDF
from utils.joint_utils import *
from utils.traj_utils import *

urdf_content = URDF.from_xml_file("../urdf/panda/panda_bullet.urdf")
link_names = get_link_names(urdf_content)
joint_names = get_joint_names(urdf_content)
base_link, tip_link = link_names[0], link_names[-1]

Unknown tag "material" in /robot[@name='panda']/link[@name='panda_link0']/collision[1]
Unknown tag "material" in /robot[@name='panda']/link[@name='panda_link1']/collision[1]
Unknown tag "material" in /robot[@name='panda']/link[@name='panda_link2']/collision[1]
Unknown tag "material" in /robot[@name='panda']/link[@name='panda_link3']/collision[1]
Unknown tag "material" in /robot[@name='panda']/link[@name='panda_link4']/collision[1]
Unknown tag "material" in /robot[@name='panda']/link[@name='panda_link5']/collision[1]
Unknown tag "material" in /robot[@name='panda']/link[@name='panda_link6']/collision[1]
Unknown tag "material" in /robot[@name='panda']/link[@name='panda_link7']/collision[1]


### Calculate task trajectory

In [19]:
from utils.algebra import *

In [21]:
## get current Q0, T0
Q0 = traj_client.get_qcur()
T0 = get_tf(tip_link, list2dict(Q0, joint_names), 
            urdf_content, from_link=base_link)

In [22]:
## make target
dT = SE3(Rot_axis(1,0), (0,0.1,0))
Ttar = np.matmul(T0, dT)

In [23]:
P0 = np.concatenate([T0[:3,3], list(reversed(Rot2zyx(T0[:3,:3])))])
P1 = np.concatenate([Ttar[:3,3], list(reversed(Rot2zyx(Ttar[:3,:3])))])

In [24]:
## Calculate s-curve trajectory in task space
se3_alg = Combined([Euclidean(3), RotationUVW()])


N_div = 100
dp = se3_alg.diff_in_alg(P0, [P1])

# make double-S curve in algebra
dp_traj = dp * (
    np.sin(np.pi * (np.arange(N_div,dtype=float)[:,np.newaxis] / N_div - 0.5)) + 1) / 2

### Calculate joint trajectory

In [26]:
dp_list = dp_traj[1:]-dp_traj[:-1]
Qi = np.copy(Q0)
joint_traj = []
for dp_i in dp_list:
    J = get_jacobian(
        tip_link, urdf_content, Qi, joint_names, ref_link=base_link)
    Qi += np.matmul(np.linalg.pinv(J), dp_i)
    joint_traj.append(np.copy(Qi))
Qfin = Qi

### check trajectory 

In [1]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(joint_traj)

### Move along joint trajectory

In [30]:
traj_client.move_joint_traj(joint_traj)

### using gripper

In [31]:
traj_client.grasp(True)

True

In [32]:
traj_client.grasp(False)

False

In [35]:
traj_client.move_gripper(0.05)

In [34]:
traj_client.move_gripper(0.01)